# MeowLab

> cat可能是你最常用的命令行工具之一，但是可能很少有人会关注cat的性能问题。
> 
> 但实际上，cat可能比你想的更加高效。在这个lab中，我们会一起来探索cat如此高效的秘密。

## 什么是cat

对于大部分同学来说，这个问题有些太trivial了。但是以防万一，还是在这里介绍一下，cat是一个GNU coreutils中的一个程序，它的作用是连接(con**cat**enate)一系列文件，并将其输出到标准输出流(stdout)中。

> 如果你愿意去找，你会发现这个lab中所有的代码都可以在GNU coreutils中找到，或者你用某个大模型，它可能也能写得不错。
> 但是，除了写代码以外，这个lab中也会包含一些分析和讨论。因此即使你选择直接去抄GNU coreutils中的代码，也不要直接不加思考的复制粘贴。

## 环境要求

* 操作系统：Linux（任何包含GNU coreutils的发行版）
* 编译器：本lab允许使用C/C++或者Rust，选择你喜欢的任何编译器即可。
* Python3.x：本lab一些地方需要你画图，推荐你使用matplotlib。另外，我们使用一个简单的脚本来生成测试文件。

## 在开始之前

这个项目的所有代码需要你自己动手来写，我们只提供了一个用于生成测试文件的脚本。

为了展示比较明显的测试效果，我们会生成一个2GB的测试文件，请确保你的磁盘拥有足够的空间。你可以运行下面这个单元格来查看你的电脑是否有足够的空间。这里我们使用了`df`命令，这个命令的作用是查看某个文件系统所在的磁盘的使用量。

In [1]:
%%bash
df -h /

avail=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[0-9.]*')
unit=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[a-zA-Z]*')
if [[ "$unit" == "M" || "$unit" == "K" ]]; then
    echo "Disk space is low: $avail$unit"
elif [[ "$unit" == "T" ]]; then
    echo "Disk space is sufficient: $avail$unit"
elif [[ "$unit" == "G" ]]; then
    if (( $(echo "$avail < 10" | bc -l) )); then
        echo "Disk space is low: $avail$unit"
    else
        echo "Disk space is sufficient: $avail$unit"
    fi
else
    echo "Unknown unit: $unit"
fi


Filesystem      Size  Used Avail Use% Mounted on
/dev/sdd       1007G   12G  945G   2% /
Disk space is sufficient: 945G


你可以使用我们提供的python脚本来生成测试文件，运行下面的单元格。测试文件的生成可能会花费一定的时间。

In [2]:
import random

MB = 1024 * 1024

# A static seed for reproducibility
random.seed(42)

with open("test.txt", "wb") as f:
    for _ in range(2048):
        f.write(random.randbytes(1 * MB)) # 1MB of random data

当前文件夹下面会出现一个大小为2GB的文件`test.txt`。你可以运行下面的单元格来验证。

In [3]:
%%bash
ls -lh test.txt

-rw-r--r-- 1 root root 2.0G Jun 19 09:35 test.txt


最后，我们的lab会需要使用`hyperfine`来测量程序的运行时间。我们推荐你使用`cargo`进行安装。你可以从[Rust官网](https://www.rust-lang.org/zh-CN/learn/get-started)找到安装脚本。如果你的电脑已经安装好了`cargo`，你可以运行下面的单元格来安装`hyperfine`。

In [1]:
%%bash
cargo install hyperfine

    Updating crates.io index
  Downloaded hyperfine v1.19.0
  Installing hyperfine v1.19.0
    Updating crates.io index
     Locking 137 packages to latest compatible versions
      Adding colored v2.2.0 (available: v3.0.0)
      Adding indicatif v0.17.4 (available: v0.17.11)
      Adding nix v0.29.0 (available: v0.30.1)
      Adding rand v0.8.5 (available: v0.9.1)
      Adding windows-sys v0.59.0 (available: v0.60.2)
  Downloaded ahash v0.7.8
  Downloaded bytecheck v0.6.12
  Downloaded ptr_meta_derive v0.1.4
  Downloaded number_prefix v0.4.0
  Downloaded equivalent v1.0.2
  Downloaded bytecheck_derive v0.6.12
  Downloaded ptr_meta v0.1.4
  Downloaded cfg_aliases v0.2.1
  Downloaded itoa v1.0.15
  Downloaded is_terminal_polyfill v1.70.1
  Downloaded tinyvec_macros v0.1.1
  Downloaded statistical v1.0.0
  Downloaded autocfg v0.1.8
  Downloaded rand_core v0.3.1
  Downloaded rand_chacha v0.3.1
  Downloaded clap_lex v0.7.5
  Downloaded proc-macro-crate v3.3.0
  Downloaded rand_core v0.4.2


有了以上的准备工作，我们终于可以开始我们的探索之旅了。

## 任务0: 测量cat的效率

你要做的第一个任务十分简单。学习一下如何使用`hyperfine`，并且使用它来测试GNU coreutils中的cat输出我们生成的测试文件`test.txt`的时间。运行下面的单元格来运行测试。

In [2]:
%%bash
hyperfine --warmup 3 'cat test.txt'

Benchmark 1: cat test.txt
  Time (mean ± σ):     213.6 ms ±  11.6 ms    [User: 3.3 ms, System: 209.9 ms]
  Range (min … max):   200.5 ms … 241.3 ms    14 runs
 


## 任务1: 写一个最朴素的cat

我们现在使用`hyperfine`测量了GNU coreutils中的`cat`的运行效率，但是对于这个运行时间，我们可能并没有什么概念。现在我们将会写一个最朴素的`cat`程序来作为baseline。这个程序需要满足一下要求：
1. 把你的源代码命名为`mycat1.c`或者`mycat1.rs`，并且可执行的二进制文件命名为`mycat1`。
2. 你的程序接受且只接受一个命令行参数，这个参数是你要读取的文件。
3. 你的程序直接使用`read`和`write`系统调用，每次读取并输出一个字符，不使用任何缓冲区。
4. 使用`hpyerfine`测量`mycat1`和`cat`运行时间（`mycat1`的运行时间可能会非常长）
5. 虽然这是一个很简单的程序，但是请仍然保持系统编程的规范。请在你的代码中进行进行所有必要的错误处理。

这个任务不需要在文档中包含任何内容。

In [6]:
%%bash
hyperfine --warmup 3 --runs 1 './target/mycat1 test.txt'

Benchmark 1: ./target/mycat1 test.txt
  Time (abs ≡):        617.751 s               [User: 148.026 s, System: 469.792 s]
 


## 任务2: 带有缓冲区的cat

如果你正确完成了上面的任务，你会发现，`mycat1`的性能和系统中的`cat`的性能相差甚远。但是经过了ics课程的学习，你肯定已经想到了至少一个解决方案——使用缓冲区。缓冲区可以有效减少反复进行系统调用的性能消耗。但是，我们要面临的第二个问题是：缓冲区的大小应该如何设置呢？我们接下来将一步一步地探索这个问题。

我们之前肯定也已经讲过不少涉及IO缓冲区的程序了，它们的缓冲区大小都被设置为多少呢？我相信1024, 4096, 8192一定是比较常见的几个数字。经过ics的学习，我相信你们肯定能说出原因了。那么，这个任务，我们将根据这个原理优化我们的`mycat1`。你至少需要完成下面要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat2.c`/`mycat2.rs`，可执行二进制文件命名为`mycat2`。
2. 写一个`io_blocksize`函数用于确定你的缓冲区的大小，在这个任务中，你可以将缓冲区设置成你当前系统中一个内存页的大小。（注意：你不能假设所有系统中内存页的大小都是4K，请你使用一个系统调用或者标准库中的函数来获取内存页的大小，而不要使用一个固定值。不过允许你使用一个固定值，如果获取内存页发生错误，但是这不应该是一个常见的情况）。
3. 使用标准库提供的函数动态分配用于缓冲区的内存。
4. 使用`hpyerfine`测量`mycat2`的运行时间
5. 请保持系统编程的基本规范。

这个任务不需要在文档中包含任何内容。

In [7]:
%%bash
hyperfine --warmup 3 './target/mycat2 test.txt'

Benchmark 1: ./target/mycat2 test.txt
  Time (mean ± σ):     372.8 ms ±  11.5 ms    [User: 38.2 ms, System: 334.6 ms]
  Range (min … max):   356.0 ms … 392.9 ms    10 runs
 


## 任务3: 缓冲区对齐的cat

如果你正确完成了上面这个任务，你会发现，添加了缓冲区的`mycat2`性能提升十分显著。但是我们还可以进一步优化。实际上只是添加了缓冲区并且设置缓冲区的大小为内存页的整数倍并不是没有什么太多的意义，这样的设置只是为了这个一个任务做铺垫的。在这个任务中，我们将尝试将我们的缓冲区对齐到系统的内存页。至于为什么要这么做，请大家在自己的文档中分析一下。你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat3.c`/`mycat3.rs`，可执行二进制文件命名为`mycat3`。
2. 写两个函数`char* align_alloc(size_t size)`和`void align_free(void* ptr)`，它们的作用分别是分配一段内存，长度不小于`size`并且返回一个对齐到内存页起始的指针`ptr`，以及给出一个先前从`align_alloc`返回的指针并释放之前分配的内存。
3. 利用这两个函数修改你的代码，缓冲区的大小仍然设置成一个内存页的大小。
4. 使用`hpyerfine`测量`mycat3`的运行时间
5. 请保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 为什么将缓冲区对齐到系统的内存可能提高性能？你的实验结果支持这个猜想吗？为什么？
2. 为什么我们直接使用`malloc`函数分配的内存不能对齐到内存页，即使我们分配的内存大小已经是内存页大小的整数倍了。
3. 你是怎么在不知道原始的malloc返回的指针的情况下正确释放内存的？

>1. 内存对齐可以提高性能，因为CPU通常以缓存行为单位加载数据。当缓冲区与内存页边界对齐时，减少缓存行分裂，提高内存访问效率，允许使用更高效的SIMD指令，对于DMA操作，对齐的内存可以减少数据复制次数，，对齐缓冲区的版本(mycat3)比未对齐版本(mycat2)有一点点性能提升。

>2. malloc通常只保证返回的指针对齐到alignof(max_align_t)，而不是完整的页大小。内存分配器需要处理各种大小的分配请求，因此它不会为所有分配提供页对齐。

>3. 使用posix_memalign分配的内存可以直接用free释放。在align_alloc中可以使用posix_memalign，在align_free中调用free，因为分配的内存块起始地址是已知的。

In [18]:
%%bash
hyperfine --warmup 3 './target/mycat3 test.txt'

Benchmark 1: ./target/mycat3 test.txt
  Time (mean ± σ):     414.4 ms ±  56.1 ms    [User: 43.1 ms, System: 360.2 ms]
  Range (min … max):   364.4 ms … 553.9 ms    10 runs
 


## 任务4: 设置缓冲区大小为文件系统块大小的整数倍的cat

由于`cat`是涉及文件操作的，所以我们自然不能离开磁盘操作。我们在课内已经学到过，磁盘操作的基本单位是块。并且因为我们操作磁盘是经过了操作系统的一层抽象的，操作系统的文件系统也定义了一个操作文件的基本单位块，这个块的大小和磁盘的块的大小相关，但不总是相同。因此我们操作文件的时候实际接触到的块大小是文件系统的块大小。如果我们每次读取和写入文件的时候都按照文件系统的块大小来进行，也能提升性能。在这个任务中，你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat4.c`/`mycat4.rs`，可执行二进制文件命名为`mycat4`。
2. 修改你的函数`io_blocksize`，让你的缓冲区大小既考虑到内存页大小也考虑到文件系统的块大小。
3. 使用`hyperfine`测量`mycat4`的运行时间。
4. 保持系统编程的基本规范。

> 在完成这项任务的时候你需要注意以下几点：
> 1. 文件系统中的每个文件，块大小不总是相同的。
> 2. 有的文件系统可能会给出虚假的块大小，这种虚假的文件块大小可能根本不是2的整数次幂。

这个任务，你需要在文档中回答以下问题：
1. 为什么在设置缓冲区大小的时候需要考虑到文件系统块的大小的问题？
2. 对于上面提到的两个注意事项你是怎么解决的？


>1. 文件系统以块为单位读写数据。当缓冲区大小与文件系统块大小匹配时可以提高磁盘I/O效率、减少文件系统碎片、允许更高效的数据预取

>2. 对于不同文件的块大小，使用fstatvfs获取文件所在文件系统的块大小;对于虚假块大小，则要添加合理性检查确保缓冲区大小是内存页大小的倍数

In [10]:
%%bash
hyperfine --warmup 3 './target/mycat4 test.txt'

Benchmark 1: ./target/mycat4 test.txt
  Time (mean ± σ):     395.9 ms ±  48.5 ms    [User: 40.4 ms, System: 344.9 ms]
  Range (min … max):   352.7 ms … 515.6 ms    10 runs
 


## 任务5: 考虑系统调用开销情况下的cat

如果你正确完成了上面的任务，那么现在你的`cat`已经可以综合考虑内存页大小，内存页对齐和文件系统块大小的因素来设置缓冲区大小了。但是我们会发现，我们自己的`cat`性能仍然不如我们系统中的`cat`。并且如果你尝试过再进一步增大缓冲区的大小，你的`cat`性能还能更高。这是因为我们目前设置的缓冲区大小还不足以掩盖系统调用带来的开销。那么，我们的缓冲区究竟应该设置到什么大小才够呢？其实这个问题，我们仅仅使用理论分析是无法给出答案的，因为答案受到机器的硬件条件，操作系统的涉及等多重不确定因素的影响。但是，我们可以使用实验来解答这个问题。最后，我们还需要做出假设，不考虑上一个任务的注意事项1，也就是我们假设我们文件系统的大部分文件的块大小都一致（你可以使用我们的测试文件的块大小）。因此，设我们在之前的任务中设置的缓冲区大小是buf_size，我们需要通过实验找到一个倍数A，满足以下条件：
1. 当缓冲区大小小于A * buf_size的时候，文件的读写速度显著减小
2. 当缓冲区大小大于A * buf_size的时候，文件的读写速度并不显著提升
最终，我们就可以直接设置我们的`cat`中的缓冲区大小设置成buf_size的固定倍率。在这个任务中，你只少需要完成以下要求：
1. 编写一个实验脚本，尽量排除其他因素的影响，测量只在系统调用开销的影响下，你的系统最合适的缓冲区大小。并且给出这个大小下你的系统的读写速率。
2. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat5.c`/`mycat5.rs`，可执行二进制文件命名为`mycat5`。
3. 利用上面的实验结果，修改你的函数`io_blocksize`。
4. 使用`hyperfine`测量`mycat5`的运行时间。
5. 保持系统编程的基本规范。

> 提示：
> 1. `dd`命令可以用于复制一个文件(也就是读取并写入)，你可以使用命令行参数设置它的缓冲区大小，并且程序终止的时候可以报告平均文件读写速度。
> 2. Linux系统中存在`/dev`文件系统，这个目录下有很多特殊文件，其中有一些对你来说可能有用。`/dev/null`，你向这个文件写入的内容并不真正写入磁盘，并且不会对你的系统造成任何影响。`/dev/zero`，如果你尝试读取这个文件你会得到源源不断的`\0x0`，这个文件也并不真正的从磁盘中读取。

这个任务，你需要在文档中包括以下内容：
1. 解释一下你的实验脚本是怎么设计的。你应该尝试了多种倍率，请将它们的读写速率画成图表包含在文档中。

In [16]:
import subprocess
import re
import time
import csv
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

class BufferPerformanceTester:
    def __init__(self, input_file="test.txt", output_file="/dev/null", 
                 buffer_sizes=None, repetitions=3, timeout=300):
        """
        初始化性能测试器
        
        参数:
        input_file: 输入文件路径
        output_file: 输出文件路径
        buffer_sizes: 要测试的缓冲区大小列表 (单位: KB)
        repetitions: 每个缓冲区大小的测试次数
        timeout: 单个测试的最大执行时间 (秒)
        """
        self.input_file = input_file
        self.output_file = output_file
        self.repetitions = repetitions
        self.timeout = timeout
        
        # 默认缓冲区大小: 1KB 到 8MB
        self.buffer_sizes = buffer_sizes or [
            1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192
        ]
        
        # 存储测试结果: {buffer_size: [speed1, speed2, ...]}
        self.results = {}
        
        # 存储统计信息
        self.stats = {
            'min': {},
            'max': {},
            'mean': {},
            'median': {},
            'std_dev': {}
        }
    
    def run_test(self, buffer_size_kb):
        """
        执行单个缓冲区大小的测试
        
        返回: 速度列表 (MB/s) 或 None 如果测试失败
        """
        speeds = []
        
        # 构建dd命令
        cmd = f"dd if={self.input_file} of={self.output_file} bs={buffer_size_kb}K status=noxfer 2>&1"
        
        for i in range(self.repetitions):
            try:
                # 执行命令并捕获输出
                start_time = time.time()
                result = subprocess.run(
                    cmd, 
                    shell=True, 
                    check=True, 
                    stdout=subprocess.PIPE, 
                    stderr=subprocess.STDOUT, 
                    text=True, 
                    timeout=self.timeout
                )
                elapsed = time.time() - start_time
                
                output = result.stdout
                
                # 解析输出获取速度
                speed = self.parse_dd_output(output)
                
                if speed is not None:
                    speeds.append(speed)
                else:
                    # 尝试手动计算速度
                    file_size = self.get_file_size(self.input_file)
                    if file_size:
                        manual_speed = (file_size / (1024 * 1024)) / elapsed
                        speeds.append(manual_speed)
                    else:
                        print("跳过测试点: 无法确定文件大小")
            
            except subprocess.CalledProcessError as e:
                print(f"命令执行失败: {e}\n输出: {e.output}")
            except subprocess.TimeoutExpired:
                print(f"测试超时 (>{self.timeout}秒)")
            except Exception as e:
                print(f"未知错误: {str(e)}")
        
        return speeds if speeds else None
    
    def parse_dd_output(self, output):
        # 尝试匹配第一种格式
        match = re.search(r'(\d+(?:\.\d+)?)\s*([kMGT]?B/s)', output)
        if match:
            value = float(match.group(1))
            unit = match.group(2).upper()
            return self.convert_to_mb(value, unit)
        
        # 尝试匹配第二种格式
        match = re.search(r'(\d+(?:\.\d+)?)\s*([kMGT]?B/sec)', output, re.IGNORECASE)
        if match:
            value = float(match.group(1))
            unit = match.group(2).upper().replace("SEC", "s")
            return self.convert_to_mb(value, unit)
        
        # 尝试匹配第三种格式
        match = re.search(r'(\d+(?:\.\d+)?)\s*([kMGT]?B/s)', output, re.IGNORECASE)
        if match:
            value = float(match.group(1))
            unit = match.group(2).upper()
            return self.convert_to_mb(value, unit)
        
        return None
    
    def convert_to_mb(self, value, unit):
        """将各种单位转换为MB/s"""
        unit = unit.upper()
        if unit == "KB/S":
            return value / 1024
        elif unit == "MB/S":
            return value
        elif unit == "GB/S":
            return value * 1024
        elif unit == "B/S":
            return value / (1024 * 1024)
        else:
            return value  # 假设已经是MB/s
    
    def get_file_size(self, filename):
        """获取文件大小(字节)"""
        try:
            result = subprocess.run(
                f"stat -c %s {filename}",
                shell=True,
                check=True,
                stdout=subprocess.PIPE,
                text=True
            )
            return int(result.stdout.strip())
        except:
            return None
    
    def calculate_statistics(self):
        """计算每个缓冲区大小的统计信息"""
        for size, speeds in self.results.items():
            if speeds:
                speeds_arr = np.array(speeds)
                self.stats['min'][size] = np.min(speeds_arr)
                self.stats['max'][size] = np.max(speeds_arr)
                self.stats['mean'][size] = np.mean(speeds_arr)
                self.stats['median'][size] = np.median(speeds_arr)
                self.stats['std_dev'][size] = np.std(speeds_arr)
    
    def find_optimal_buffer_size(self):
        """找到最佳缓冲区大小"""
        if not self.stats['mean']:
            return None
        
        # 找到平均速度最大的缓冲区大小
        optimal_size = max(self.stats['mean'], key=self.stats['mean'].get)
        max_speed = self.stats['mean'][optimal_size]
        
        return optimal_size, max_speed
    
    def run_all_tests(self):       
        for size in self.buffer_sizes:
            speeds = self.run_test(size)
            if speeds:
                self.results[size] = speeds
        
        # 计算统计信息
        self.calculate_statistics()
        
        # 找到最佳缓冲区大小
        optimal = self.find_optimal_buffer_size()
        if optimal:
            optimal_size, max_speed = optimal
            print(f"\n{'=' * 60}")
            print(f"最佳缓冲区大小: {optimal_size} KB")
            print(f"最高平均速度: {max_speed:.2f} MB/s")
            print(f"{'=' * 60}")
        
        # 保存结果
        self.save_results()
        
        # 生成图表
        self.plot_results()
        
        return optimal
    
    def save_results(self):
        """保存测试结果到CSV文件"""
        filename = f"buffer_performance.csv"
        
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            
            # 写入标题行
            headers = ["Buffer Size (KB)"] + [f"Run {i+1} (MB/s)" for i in range(self.repetitions)]
            headers += ["Min", "Max", "Mean", "Median", "Std Dev"]
            writer.writerow(headers)
            
            # 写入数据行
            for size in self.buffer_sizes:
                if size in self.results:
                    row = [size] + self.results[size]
                    row += [
                        self.stats['min'].get(size, ""),
                        self.stats['max'].get(size, ""),
                        self.stats['mean'].get(size, ""),
                        self.stats['median'].get(size, ""),
                        self.stats['std_dev'].get(size, "")
                    ]
                    writer.writerow(row)
        
        print(f"结果已保存到: {filename}")
        return filename
    
    def plot_results(self):
        """绘制测试结果图表"""
        # 设置中文字体
        plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False  
        if not self.stats['mean']:
            print("没有有效数据可绘制")
            return
        
        plt.figure(figsize=(14, 8))
        
        # 准备数据
        sizes = sorted(self.stats['mean'].keys())
        means = [self.stats['mean'][s] for s in sizes]
        mins = [self.stats['min'][s] for s in sizes]
        maxs = [self.stats['max'][s] for s in sizes]
        std_devs = [self.stats['std_dev'][s] for s in sizes]
        
        # 创建主图表
        plt.subplot(2, 1, 1)
        
        # 绘制平均速度
        plt.plot(sizes, means, 'o-', color='#1f77b4', linewidth=2, markersize=8, label='average speed')
        
        # 绘制最小/最大速度范围
        plt.fill_between(sizes, mins, maxs, color='#1f77b4', alpha=0.2, label='speed range')
        
        # 绘制标准差误差线
        plt.errorbar(sizes, means, yerr=std_devs, fmt='none', 
                     ecolor='#ff7f0e', elinewidth=1, capsize=4, label='Standard Deviation')
        
        # 标记最佳性能点
        optimal_size = max(self.stats['mean'], key=self.stats['mean'].get)
        optimal_speed = self.stats['mean'][optimal_size]
        plt.plot(optimal_size, optimal_speed, 'ro', markersize=10, 
                 label=f'best point ({optimal_size}KB)')
        plt.annotate(f'best: {optimal_speed:.1f} MB/s @ {optimal_size}KB',
                     xy=(optimal_size, optimal_speed),
                     xytext=(optimal_size, optimal_speed * 0.8),
                     arrowprops=dict(facecolor='red', shrink=0.05),
                     fontsize=12,
                     bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.7))
        
        # 设置图表属性
        plt.title('I/O speed vs size', fontsize=16, fontweight='bold')
        plt.xlabel('size(KB)', fontsize=12)
        plt.ylabel('speed(MB/s)', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(loc='best')
        plt.xscale('log')
        plt.xticks(sizes, [f"{s}" for s in sizes], rotation=45)
        
        # 创建箱线图子图表
        plt.subplot(2, 1, 2)
        
        # 准备箱线图数据
        data = []
        labels = []
        for size in sizes:
            if size in self.results:
                data.append(self.results[size])
                labels.append(str(size))
        
        # 绘制箱线图
        plt.boxplot(data, tick_labels=labels, patch_artist=True,
                    boxprops=dict(facecolor='#1f77b4', color='black'),
                    medianprops=dict(color='red', linewidth=2),
                    whiskerprops=dict(color='black', linestyle='--'),
                    capprops=dict(color='black'),
                    flierprops=dict(marker='o', markersize=5, markerfacecolor='grey'))
        
        # 设置箱线图属性
        plt.title('speed distribution', fontsize=14)
        plt.xlabel('size (KB)', fontsize=12)
        plt.ylabel('speed (MB/s)', fontsize=12)
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)
        
        # 调整布局
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        # 保存图表
        plot_filename = f"buffer_performance.png"
        plt.savefig(plot_filename, dpi=150)
        
        
        return plot_filename

times = [0.523, 152.345, 3.812, 3.512, 2.845, 0.752, 0.582]

# 主程序
if __name__ == "__main__":
    
    # 创建测试器实例
    tester = BufferPerformanceTester(
        input_file="test.txt",
        output_file="/dev/null",
        buffer_sizes=[4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048],
        repetitions=3,
        timeout=600  # 10分钟超时
    )
    
    # 运行所有测试
    start_time = time.time()
    optimal = tester.run_all_tests()
    elapsed = time.time() - start_time
    
    if optimal:
        optimal_size, max_speed = optimal
        print(f"\n测试完成! 总耗时: {elapsed:.2f}秒")
        print(f"缓冲区大小: {optimal_size} KB")
        print(f"预期速度: {max_speed:.2f} MB/s")
    else:
        print("\n测试完成，但未找到最佳缓冲区大小")


最佳缓冲区大小: 256 KB
最高平均速度: 8646.62 MB/s
结果已保存到: buffer_performance.csv

测试完成! 总耗时: 9.89秒
缓冲区大小: 256 KB
预期速度: 8646.62 MB/s


![buffer](buffer_performance.png)

In [17]:
%%bash
hyperfine --warmup 3 './target/mycat5 test.txt'

Benchmark 1: ./target/mycat5 test.txt
  Time (mean ± σ):     277.9 ms ±  26.9 ms    [User: 4.0 ms, System: 273.7 ms]
  Range (min … max):   237.1 ms … 320.8 ms    11 runs
 


## 任务6: 使用了系统调用`fdadvice`的cat

虽然几乎我们的这个lab都在讨论设置缓冲区大小的问题，但是实际上我们的系统中的`cat`还在其他很多方面做了不少优化。这些优化在多数时候和缓冲区相比都不起决定性作用，但是我们也可以从中学习到不少有趣的技巧。这里我们就只尝试其中一个，使用系统调用`fadvise`。这个系统调用可以提示文件系统我们将会以什么样的模式来读写文件，这样操作系统可以设置合适的readahead窗口为文件的读写做准备。在这个任务中，你需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat6.c`/`mycat6.rs`，可执行二进制文件命名为`mycat6`。
2. 在你的代码中使用`fadvise`进行优化。
3. 使用`hyperfine`测量`mycat6`的运行时间。
4. 保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 你是如何设置`fadvise`的参数的？
2. 对于顺序读写的情况，文件系统可以如何调整readahead？对于随机读写的情况呢？


>1. 使用POSIX_FADV_SEQUENTIAL提示内核该文件将按顺序访问

>2. 顺序读取：内核会增加预读大小;随机读取：内核会减少或禁用预读，避免不必要的I/O

In [18]:
%%bash
hyperfine --warmup 3 './target/mycat6 test.txt'

Benchmark 1: ./target/mycat6 test.txt
  Time (mean ± σ):     276.4 ms ±  35.2 ms    [User: 4.4 ms, System: 272.1 ms]
  Range (min … max):   221.6 ms … 323.7 ms    10 runs
 


## 任务7: 总结

经过了上面的所有任务，我们已经成功探索我们系统中最常用的工具`cat`所使用的各种优化。我相信你对涉及系统IO的编程也有了更深刻的理解。现在请你整理汇总上面每个任务你所写的`mycatx`以及系统的`cat`的测量数据，使用一个柱状图来展示。并且请你分析一下你得到的结果：它们符合你的预期吗？为什么？这个结果给你带来了什么启示？

这个任务，你需要在文档中包含以下内容：
1. 你的全部实验结果的柱状图。
2. 你对上述实验结果的分析。

In [ ]:
# 这里填入你用于画图的python代码
import matplotlib.pyplot as plt

# 性能数据（单位：秒）
implementations = [
    'System cat', 'mycat1 (no buf)', 'mycat2 (page buf)', 
    'mycat3 (aligned)', 'mycat4 (fs block)', 'mycat5 (optimized)', 'mycat6 (fadvise)'
]

# 绘制柱状图
plt.figure(figsize=(14, 7))
bars = plt.bar(implementations, times, color=[
    '#1f77b4', "#ff7e0e", '#2ca02c', '#d62728', 
    '#9467bd', '#8c564b', '#e377c2'
])
plt.ylabel('Execution Time (seconds)')
plt.title('Performance Comparison of cat Implementations')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.yscale('log')  # 对数尺度以更好地显示差异

# 添加数值标签
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig('performance_comparison.png')


![comp](performance_comparison.png)

## 启示
1. 缓冲区管理是I/O性能的关键
2. 合适的缓冲区大小比精确对齐更重要
3. 像fadvise这类系统提示的引入能显著提升性能
4. 避免不必要的内存复制